In [3]:
import numpy as np
import hdbscan
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors

## Core distance calculation
Core distances are essential for density-based clustering because they capture how densely packed a point's neighborhood is:
If a point has a small core distance, it is surrounded by many close neighbors and is likely in a dense region of the data.


If a point has a large core distance, it is more isolated or lies on the edge of a cluster or even in noise.


In the context of HDBSCAN, core distances help determine which points should be considered core points, and they are used in calculating mutual reachability distance, which takes into account both the density of the points and their distance.


it helps distinguish between core points (in dense regions) and border or noise points (in sparser regions).


In [4]:
## we use the NearestNeighbors class to find the nearest neighbors for each point in the dataset. 
## n_neighbors=min_samples: Specifies how many nearest neighbors to compute for each point. 
## In DBSCAN or HDBSCAN, this corresponds to min_samples, which is the number of neighbors required 
## for a point to be considered a core point.


def core_distances(X, min_samples):
    """Compute the core distance for each point in X"""
    neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X)
    distances, indices = neighbors.kneighbors(X)
    # The core distance is the distance to the min_samples-th (kth)nearest neighbor
    # Taking maximum distance as core distance
    core_dist = distances[:, -1]
    return core_dist


In [5]:
def mutual_reachability_distance(X, core_dist):
    """Compute the mutual reachability distance matrix"""
    # Compute pairwise Euclidean distances
    pairwise_dist = pairwise_distances(X)
    
    # Mutual reachability distance: max(core_dist[i], core_dist[j], pairwise_dist[i,j])
    reachability_dist = np.maximum(pairwise_dist, core_dist[:, np.newaxis])
    reachability_dist = np.maximum(reachability_dist, core_dist[np.newaxis, :])
    
    return reachability_dist

In [6]:
def modified_silhouette_score(X, labels, min_samples):
    """Compute the modified Silhouette Score using mutual reachability distance"""
    # Filter out noise points (label == -1)
    core_mask = labels != -1
    X_core = X[core_mask]
    labels_core = labels[core_mask]
    
    # Compute core distances for core points
    core_dist = core_distances(X_core, min_samples)
    
    # Compute the mutual reachability distance matrix
    reachability_dist = mutual_reachability_distance(X_core, core_dist)
    
    silhouette_vals = []
    
    for i, label in enumerate(labels_core):
        # Intra-cluster distance a(i)
        same_cluster_mask = (labels_core == label)
        a_i = np.mean(reachability_dist[i, same_cluster_mask])
        
        # Inter-cluster distance b(i) (nearest cluster)
        b_i = np.inf
        for other_label in np.unique(labels_core):
            if other_label != label:
                other_cluster_mask = (labels_core == other_label)
                b_i = min(b_i, np.mean(reachability_dist[i, other_cluster_mask]))
        
        # Silhouette score for point i
        s_i = (b_i - a_i) / max(a_i, b_i)
        silhouette_vals.append(s_i)
    
    # Return the average silhouette score
    return np.mean(silhouette_vals)


In [7]:
# Sample data
from sklearn.datasets import make_blobs

# Generate synthetic data with noise
X, _ = make_blobs(n_samples=500, centers=5, cluster_std=0.6, random_state=42)


In [8]:
# Apply HDBSCAN clustering
hdb = hdbscan.HDBSCAN(min_cluster_size=10, gen_min_span_tree=True)
labels = hdb.fit_predict(X)


In [9]:
# Calculate the modified Silhouette Score
score = modified_silhouette_score(X, labels, min_samples=10)
print(f"Modified Silhouette Score using Mutual Reachability Distance: {score}")

Modified Silhouette Score using Mutual Reachability Distance: 0.7999548521199888
